<a href="https://colab.research.google.com/github/seansmith2600/H6060-Colab-Notebooks/blob/main/H6060_Masters_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages
import os
import six
import logging
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime
from pandas.plotting import table
from matplotlib.font_manager import FontProperties

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Disable font warnings
logging.getLogger('matplotlib.font_manager').disabled = True

# Mount your Google Drive to Collaboratory
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Getter functions

def get_notebook_directory_path_prefix():
  return 'gdrive/My Drive/Colab Notebooks'

# Common report columns
def get_security_tool_name():
  return 'Security Tool Name'

def get_experiment_project_name():
  return 'Experiment Project Name'

def get_experiment_github_branch():
  return 'Experiment GitHub Branch'

def get_experiment_runner_operating_system():
  return 'Experiment Runner Operating System'

# SCA report columns
def get_cve_id():
  return 'CVE ID'

def get_severity():
  return 'Severity'

def get_component_name():
  return 'Component Name'

def get_component_scope():
  return 'Component Scope'

def get_dependency_name():
  return 'Dependency Name'

def get_osv_vulnerability_cve():
  return 'OSV Vulnerability CVE'

def get_osv_vulnerability_severity():
  return 'OSV Vulnerability Severity'

# SAST report columns
def get_cwe_id():
  return 'CWE ID'

def get_confidence():
  return 'Confidence'

def get_owasp_top_ten():
  return 'OWASP Top 10'

def get_mitre_top_twenty_five():
  return 'Mitre Top 25'

def get_main_source_code():
  return 'Main Source Code'

# Runner OS
def get_os_macos():
  return 'macOS 14'

def get_os_ubuntu():
  return 'Ubuntu 22.04'

def get_os_windows():
  return 'Windows 2022'

# SCA Tools
def get_grype():
  return 'grype'

def get_snyk():
  return 'snyk'

def get_dependabot():
  return 'dependabot'

def get_owasp_dependency_check():
  return 'OWASP Dependency Check'

def get_owasp_dependency_check_alias():
  return 'odc'

def get_eclipse_steady():
  return 'Eclipse Steady'

def get_eclipse_steady_alias():
  return 'es'

# SAST Tools
def get_codeql():
  return 'CodeQL'

def get_deepsource():
  return 'DeepSource'

def get_horusec():
  return 'horusec'

def get_semgrep():
  return 'semgrep'

def get_snyk_code():
  return 'snyk code'

def get_sonarqube():
  return 'SonarQube'

In [ ]:
# Create directories for saved chart reports and images
current_date = datetime.today().strftime('%Y_%m_%d')

Path(f'{get_notebook_directory_path_prefix()}/excel_reports/{current_date}').mkdir(parents=True, exist_ok=True)
Path(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/open_source_vulnerability').mkdir(parents=True, exist_ok=True)

for sca_tool in [get_dependabot(), get_eclipse_steady_alias(), get_grype(), get_owasp_dependency_check_alias(), get_snyk()]:
    Path(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{sca_tool}').mkdir(parents=True, exist_ok=True)
    Path(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/precision/{sca_tool}').mkdir(parents=True, exist_ok=True)


for sast_tool in [get_codeql().lower(), get_deepsource().lower(), get_horusec(), get_semgrep(), get_snyk_code().replace(' ', '_'), get_sonarqube().lower()]:
    Path(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{sast_tool}').mkdir(parents=True, exist_ok=True)

# Read CSV reports
df_sca_direct_dependencies_report       = pd.read_csv(f'{get_notebook_directory_path_prefix()}/experiment_data/sca_direct_dependencies.csv')
df_sca_combined_report                  = pd.read_csv(f'{get_notebook_directory_path_prefix()}/experiment_data/sca_combined_report.csv')
df_sast_combined_report                 = pd.read_csv(f'{get_notebook_directory_path_prefix()}/experiment_data/sast_combined_report.csv')
df_github_repository_tag_report         = pd.read_csv(f'{get_notebook_directory_path_prefix()}/experiment_data/github_repository_tag_report.csv')
df_github_repository_information_report = pd.read_csv(f'{get_notebook_directory_path_prefix()}/experiment_data/github_repository_information_report.csv')
df_cyclonedx_sbom_osv_report            = pd.read_csv(f'{get_notebook_directory_path_prefix()}/experiment_data/cyclonedx_sbom_osv_report.csv')

# Define dataframe containing rows of each SCA tool
df_dependabot_data             = df_sca_combined_report.loc[df_sca_combined_report[get_security_tool_name()].str.upper() == get_dependabot().upper()].copy()
df_eclipse_steady_data         = df_sca_combined_report.loc[df_sca_combined_report[get_security_tool_name()].str.upper() == get_eclipse_steady().upper()].copy()
df_grype_data                  = df_sca_combined_report.loc[df_sca_combined_report[get_security_tool_name()].str.upper() == get_grype().upper()].copy()
df_owasp_dependency_check_data = df_sca_combined_report.loc[df_sca_combined_report[get_security_tool_name()].str.upper() == get_owasp_dependency_check().upper()].copy()
df_snyk_data                   = df_sca_combined_report.loc[df_sca_combined_report[get_security_tool_name()].str.upper() == get_snyk().upper()].copy()

# Define dataframe containing rows of each SAST tool
df_codeql_data     = df_sast_combined_report.loc[df_sast_combined_report[get_security_tool_name()].str.upper() == get_codeql().upper()].copy()
df_deepsource_data = df_sast_combined_report.loc[df_sast_combined_report[get_security_tool_name()].str.upper() == get_deepsource().upper()].copy()
df_horusec_data    = df_sast_combined_report.loc[df_sast_combined_report[get_security_tool_name()].str.upper() == get_horusec().upper()].copy()
df_semgrep_data    = df_sast_combined_report.loc[df_sast_combined_report[get_security_tool_name()].str.upper() == get_semgrep().upper()].copy()
df_snyk_code_data  = df_sast_combined_report.loc[df_sast_combined_report[get_security_tool_name()].str.upper() == get_snyk_code().upper()].copy()
df_sonarqube_data  = df_sast_combined_report.loc[df_sast_combined_report[get_security_tool_name()].str.upper() == get_sonarqube().upper()].copy()

In [ ]:
# Create excel writer instance
excel_report_path = f'{get_notebook_directory_path_prefix()}/excel_reports/{current_date}/masters_report.xlsx'

# Delete file if exists
try:
    os.remove(excel_report_path)
except OSError:
    pass

def save_excel_sheet(df, sheetname, index=False):
    # Create file if it does not exist
    if not os.path.exists(excel_report_path):
        df.to_excel(excel_report_path, sheet_name=sheetname, index=index)

    # Otherwise, add a sheet. Overwrite if there exists one with the same name.
    else:
        with pd.ExcelWriter(excel_report_path, engine='openpyxl', if_sheet_exists='replace', mode='a') as writer:
            df.to_excel(writer, sheet_name=sheetname, index=index)

def append_to_excel_sheet(df, sheetname, index=False):
    wb = load_workbook(filename = excel_report_path)
    ws = wb[sheetname]
    for r in dataframe_to_rows(df, index=False, header=False):  # No index and don't append the column headers
        ws.append(r)
    wb.save(excel_report_path)

<hr style="border:2px solid gray">

## Get SCA, SAST and Experiment Project Names

In [ ]:
# Get security testing tool and experiment project names
sca_tool_names           = df_sca_combined_report[get_security_tool_name()].unique()
sast_tool_names          = df_sast_combined_report[get_security_tool_name()].unique()
experiment_project_names = df_cyclonedx_sbom_osv_report[get_experiment_project_name()].unique()

df_sca_tool_names           = pd.DataFrame(sca_tool_names)
df_sast_tool_names          = pd.DataFrame(sast_tool_names)
df_experiment_project_names = pd.DataFrame(experiment_project_names)

df_sca_tool_names.rename(columns={0 :'SCA Tool Names'}, inplace=True)
df_sast_tool_names.rename(columns={0:'SAST Tool Names'}, inplace=True)
df_experiment_project_names.rename(columns={0 :'Experiment Project Names'}, inplace=True)

<hr style="border:2px solid gray">

## MatPlotLib - Table Design

In [ ]:
# MatPlot table functions
def render_matplotlib_table(data, table_title, table_title_size=22, col_width=5.0, row_height=0.625,
                            font_size=14, cellLoc='center', header_color='#40466e', row_colors=['#f1f1f2', 'w'],
                            edge_color='w', bbox=[0, 0, 1, 1], header_columns=0, ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    ax.axis([0, 1, data.shape[0], -1])

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, cellLoc=cellLoc, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    ax.set_title(table_title, weight='bold', size=table_title_size, color='crimson', pad=20)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0] % len(row_colors) ])

            # Change cell color here for matching values
            cell_value = str(cell.get_text()).replace('Text(0, 0, ', '').replace(')','')
            if 'match_' in cell_value:
                cell.get_text().set_color('#006400')
                cell.get_text().set_text(cell_value.replace('match_', '').replace('\'',''))
                cell.set_text_props(fontproperties=FontProperties(weight='bold'))
                cell.set_fontsize(16)
    return ax

def set_row_edge_color(ax, row, color):
    ax.axhline(y=row, color=color)

<hr style="border:2px solid gray">

## DataFrame Report - Reusable Functions

In [ ]:
# ==================
# OSV Summary Report
# ==================
#
# Creates an OSV summary report containing the following columns:
#  - PROJECT: Experiment project name
#  - TAG: Experiment tag name
#  - REQUIRED DEPENDENCY: Count of component scope set as Required
#  - OPTIONAL DEPENDENCY: Count of component scope set as Optional
#  - TOTAL CVE: Count of all CVE IDs reported by OSV API based on CycloneDX SBOM
#  - UNIQUE CVE: Count of unique CVE IDs reported by OSV API based on CycloneDX SBOM
#  - CRITICAL SEVERITY: Count of all severities marked as Critical
#  - HIGH SEVERITY: Count of all severities marked as High
#  - MEDIUM SEVERITY: Count of all severities marked as Medium
#  - LOW SEVERITY: Count of all severities marked as Low
#  - UNKNOWN SEVERITY: Count of all severities marked as Unknown
def create_osv_summary_report(source_df):
    # Retrieve columns of interest from source dataframe
    new_df = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_component_scope(), get_osv_vulnerability_cve(), get_osv_vulnerability_severity()]].copy()

    # Create new columns with calculated data
    new_df['Required Dependency'] = new_df.groupby(get_experiment_project_name())[get_component_scope()].transform(lambda x: x.value_counts().loc[x.unique()].get('required', 0))
    new_df['Optional Dependency'] = new_df.groupby(get_experiment_project_name())[get_component_scope()].transform(lambda x: x.value_counts().loc[x.unique()].get('optional', 0))

    new_df['Total CVE']  = new_df.groupby(get_experiment_project_name())[get_osv_vulnerability_cve()].transform('count')
    new_df['Unique CVE'] = new_df.groupby(get_experiment_project_name())[get_osv_vulnerability_cve()].transform('nunique')

    new_df['Critical Severity'] = new_df.groupby(get_experiment_project_name())[get_osv_vulnerability_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('CRITICAL', 0))
    new_df['High Severity']     = new_df.groupby(get_experiment_project_name())[get_osv_vulnerability_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('HIGH',     0))
    new_df['Medium Severity']   = new_df.groupby(get_experiment_project_name())[get_osv_vulnerability_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('MEDIUM',   0))
    new_df['Low Severity']      = new_df.groupby(get_experiment_project_name())[get_osv_vulnerability_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('LOW',      0))
    new_df['Unknown Severity']  = new_df.groupby(get_experiment_project_name())[get_osv_vulnerability_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('UNKNOWN',  0))

    # Remove unused columns
    new_df = new_df.drop(get_osv_vulnerability_cve(), axis=1)
    new_df = new_df.drop(get_osv_vulnerability_severity(), axis=1)
    new_df = new_df.drop(get_component_scope(), axis=1)

    # If no CVE IDs were reported for a project, add a row containing 0 count values
    for project_name in experiment_project_names:
        if project_name not in new_df[get_experiment_project_name()].unique():
            # Get Experiment Project Tag
            project_tag_df = df_cyclonedx_sbom_osv_report.loc[df_cyclonedx_sbom_osv_report[get_experiment_project_name()] == project_name]
            project_tag = ''.join(project_tag_df[get_experiment_github_branch()].unique())

            # Add row to end of dataframe and increase index
            new_index = new_df.index.max() + 1
            new_df.loc[new_index] = [project_name, project_tag, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    # Drop duplicate values
    new_df = new_df.drop_duplicates(ignore_index=True)

    # Change column names
    new_df = new_df.rename(
        columns = {
            get_experiment_project_name() : 'Project',
            get_experiment_github_branch() : 'Tag',
        }
    )

    # Sort column by Project
    new_df = new_df.sort_values('Project')

    # Reset index
    new_df.reset_index(drop=True, inplace=True)

    return new_df

In [ ]:
# ==================
# SCA Summary Report
# ==================
#
# Creates a SCA tool summary report containing the following columns:
#  - PROJECT: Experiment project name
#  - TAG: Experiment tag name
#  - TOTAL CVE: Count of all CVE IDs reported by SCA tool
#  - UNIQUE CVE: Count of unique CVE IDs reported by SCA tool
#  - CRITICAL SEVERITY: Count of all severities marked as Critical
#  - HIGH SEVERITY: Count of all severities marked as High
#  - MEDIUM SEVERITY: Count of all severities marked as Medium
#  - LOW SEVERITY: Count of all severities marked as Low
#  - UNKNOWN SEVERITY: Count of all severities marked as Unknown
def create_sca_summary_report(source_df, operating_system, is_eclipse_steady=False):
    if operating_system is not None:
        # Get rows matching operating system
        source_df = source_df.loc[source_df[get_experiment_runner_operating_system()].str.upper() == operating_system.upper()]
        new_df    = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_experiment_runner_operating_system(), get_cve_id(), get_severity()]].copy()
    else:
        new_df = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_cve_id(), get_severity()]].copy()

    # Create new columns with calculated data
    total_cve_count         = new_df.groupby(get_experiment_project_name())[get_cve_id()].transform('count')
    unique_cve_count        = new_df.groupby(get_experiment_project_name())[get_cve_id()].transform('nunique').astype(int).astype(str)
    critical_severity_count = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('CRITICAL', 0))
    high_severity_count     = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('HIGH',     0))
    medium_severity_count   = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('MEDIUM',   0))
    low_severity_count      = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('LOW',      0))
    unknown_severity_count  = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('UNKNOWN',  0))

    if operating_system is None:
        # Ignore duplicate results per OS
        total_cve_count         /= 3
        critical_severity_count /= 3
        high_severity_count     /= 3
        medium_severity_count   /= 3
        low_severity_count      /= 3
        unknown_severity_count  /= 3

    new_df['Total CVE']  = total_cve_count
    new_df['Unique CVE'] = unique_cve_count

    new_df['Critical Severity'] = critical_severity_count
    new_df['High Severity']     = high_severity_count
    new_df['Medium Severity']   = medium_severity_count
    new_df['Low Severity']      = low_severity_count
    new_df['Unknown Severity']  = unknown_severity_count

    # Convert from float -> int -> string
    new_df['Total CVE']         = new_df['Total CVE'].astype(int).astype(str)
    new_df['Critical Severity'] = new_df['Critical Severity'].astype(int).astype(str)
    new_df['High Severity']     = new_df['High Severity'].astype(int).astype(str)
    new_df['Medium Severity']   = new_df['Medium Severity'].astype(int).astype(str)
    new_df['Low Severity']      = new_df['Low Severity'].astype(int).astype(str)
    new_df['Unknown Severity']  = new_df['Unknown Severity'].astype(int).astype(str)

    # Remove unused columns
    new_df = new_df.drop(get_cve_id(), axis=1)
    new_df = new_df.drop(get_severity(), axis=1)

    # If no CVE IDs were reported for a project, add a row containing 0 count values
    for project_name in experiment_project_names:
        if project_name not in new_df[get_experiment_project_name()].unique():
            if is_eclipse_steady and 'JAVASCRIPT' in project_name.upper():
                # Eclipse Steady does not support JavaScript
                continue
            else:
                # Get Experiment Project Tag
                project_tag_df = df_cyclonedx_sbom_osv_report.loc[df_cyclonedx_sbom_osv_report[get_experiment_project_name()] == project_name]
                project_tag = ''.join(project_tag_df[get_experiment_github_branch()].unique())

                # Add row to end of dataframe and increase index
                new_index = new_df.index.max() + 1
                if operating_system is not None:
                    new_df.loc[new_index] = [project_name, project_tag, operating_system, 0, 0, 0, 0, 0, 0, 0]
                else:
                    new_df.loc[new_index] = [project_name, project_tag, 0, 0, 0, 0, 0, 0, 0]

    # Drop duplicate values
    new_df = new_df.drop_duplicates(ignore_index=True)

    # Set column names
    new_df = new_df.rename(
        columns = {
            get_experiment_project_name() : 'Project',
            get_experiment_github_branch() : 'Tag',
            get_experiment_runner_operating_system() : 'Runner OS'
        }
    )

    # Sort column by Project
    new_df = new_df.sort_values('Project')

    # Reset index
    new_df.reset_index(drop=True, inplace=True)

    return new_df

In [ ]:
# ===================================
# OSV & SCA Summary Comparison Report
# ===================================
#
# Creates a combined OSV and SCA tool summary report containing the following columns:
#  - PROJECT: Experiment project name
#  - TAG: Experiment tag name
#  - RUNNER OS: Experiment operating system
#  - MATCHING CVE ID: Tool CVE ID that matches OSV report
#  - MATCHING SEVERITY: Tool Severity that matches OSV report
def create_sca_cve_id_report(source_df, operating_system):
    if operating_system is not None:
        # Get rows matching operating system
        source_df = source_df.loc[source_df[get_experiment_runner_operating_system()].str.upper() == operating_system.upper()]
        new_df    = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_experiment_runner_operating_system(), get_cve_id(), get_severity(), get_component_name()]].copy()
    else:
        new_df = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_cve_id(), get_severity(), get_component_name()]].copy()

    # Sort column by Project
    new_df = new_df.sort_values(get_experiment_project_name())

    # Reset index
    new_df.reset_index(drop=True, inplace=True)
    return new_df

def create_sca_osv_cve_comparison_report(source_df, osv_df, operating_system):
    new_osv_df = osv_df[[get_experiment_project_name(), get_experiment_github_branch(), get_osv_vulnerability_cve(), get_osv_vulnerability_severity(), get_component_name()]].copy()

    # Get SCA dataframe for all CVE IDs discovered
    new_sca_df = create_sca_cve_id_report(source_df, operating_system)

    # Match column names in both dataframes
    new_osv_df = new_osv_df.rename(columns={get_osv_vulnerability_cve(): get_cve_id()})

    # Find rows in both dataframes that match
    new_sca_df = pd.merge(new_sca_df, new_osv_df, on=[get_experiment_project_name(), get_experiment_github_branch(), get_cve_id()], how='left', indicator='Match_CVE')
    new_sca_df['Match_CVE'] = np.where(new_sca_df.Match_CVE == 'both', new_sca_df[get_cve_id()], False)
    new_sca_df['Match_Severity'] = np.where(new_sca_df[get_osv_vulnerability_severity()] == new_sca_df[get_severity()], new_sca_df[get_severity()], "MISMATCH")

    # Check if OSV component name is in SCA tool component name
    # For Java, naming component naming conventions can be different across tools
    new_sca_df['Match_Component'] = np.where([str(row[0]) in str(row[1]) for row in zip(new_sca_df[f'{get_component_name()}_y'], new_sca_df[f'{get_component_name()}_x'])], new_sca_df[f'{get_component_name()}_y'], "MISMATCH")

    # Drop non-matching CVE IDs
    new_sca_df = new_sca_df[new_sca_df.Match_CVE != 0]

    # Drop non-matching Components
    new_sca_df = new_sca_df[new_sca_df.Match_Component != "MISMATCH"]

    # Set column names
    new_sca_df= new_sca_df.rename(
        columns = {
            get_experiment_project_name() : 'Project',
            get_experiment_github_branch() : 'Tag',
            get_experiment_runner_operating_system() : 'Runner OS',
            'Match_Component' : 'Matching Component Name',
            'Match_CVE' : 'Matching CVE ID',
            'Match_Severity' : 'Matching Severity'
        }
    )

    # Remove unused columns
    new_sca_df = new_sca_df.drop(get_cve_id(), axis=1)
    new_sca_df = new_sca_df.drop(get_severity(), axis=1)
    new_sca_df = new_sca_df.drop(get_osv_vulnerability_severity(), axis=1)
    new_sca_df = new_sca_df.drop(f'{get_component_name()}_x', axis=1)
    new_sca_df = new_sca_df.drop(f'{get_component_name()}_y', axis=1)

    # Drop duplicate values
    new_sca_df = new_sca_df.drop_duplicates(ignore_index=True)

    # Drop duplicate columns
    new_sca_df = new_sca_df.loc[:,~new_sca_df.columns.duplicated()].copy()

    # Reset index
    new_sca_df.reset_index(drop=True, inplace=True)
    return new_sca_df

In [ ]:
# =======================
# SCA Dependencies Report
# =======================
#
# Creates multiple dependency reports for an SCA tool containing direct and transitive dependencies per project
#
# Creates a direct and transitive dependency report for an SCA too report containing the following columns:
#  - PROJECT: Experiment project name
#  - TAG: Experiment tag name
#  - RUNNER OS: Experiment operating system
#  - CVE ID: Vulnerability ID detected by SCA tool
#  - COMPONENT_NAME: Name of component associated with CVE ID
#  - DIRECT DEPENDENCY: Boolean to determine if direct dependency
#  - TRANSITIVE DEPENDENCY: Boolean to determine if transitive dependency
def create_sca_dependencies_reports(source_df, operating_system, tool_name, includes_initial_data=False):
    if operating_system is not None:
        # Get rows matching operating system
        source_df = source_df.loc[source_df[get_experiment_runner_operating_system()].str.upper() == operating_system.upper()]
        new_df    = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_experiment_runner_operating_system(), get_cve_id(), get_component_name()]].copy()
    else:
        new_df = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_cve_id(), get_component_name()]].copy()


    # Drop duplicate values
    new_df = new_df.drop_duplicates(ignore_index=True)

    # Set column names
    new_df = new_df.rename(
        columns = {
            get_experiment_project_name() : 'Project',
            get_experiment_github_branch() : 'Tag',
            get_experiment_runner_operating_system() : 'Runner OS'
        }
    )

    for project_name in experiment_project_names:
        # Get dependency list for each project
        df_project_dependencies = df_sca_direct_dependencies_report.loc[df_sca_direct_dependencies_report[get_experiment_project_name()].str.upper() == project_name.upper()]
        dependency_list = df_project_dependencies[get_dependency_name()].tolist()

        # Get rows of matching project
        sca_project_df = new_df.loc[new_df['Project'].str.upper() == project_name.upper()].copy()

        # Determine if it is a direct dependency, if it is exists in project dependency list
        sca_project_df['Direct Dependency'] = np.where(sca_project_df[get_component_name()].apply(lambda x: any(x in s for s in dependency_list)), True, False)
        # If not, then it is a transitive dependency
        sca_project_df['Transitive Dependency'] = ~sca_project_df['Direct Dependency']

        # Save each project report
        if includes_initial_data:
            save_excel_sheet(sca_project_df, f'{tool_name}_dependency_report')
            includes_initial_data = False
        else:
            append_to_excel_sheet(sca_project_df, f'{tool_name}_dependency_report')

    return None

In [ ]:
# ===================================
# OSV & SCA Summary Comparison Report
# ===================================
#
# Creates a report containing the following columns
#  - PROJECT: Experiment project name
#  - TAG: Experiment tag name
#  - TOTAL CVE: Count of all CVE IDs reported by SCA tool
#  - UNIQUE CVE: Count of unique CVE IDs reported by SCA tool
#  - CRITICAL SEVERITY: Count of all severities marked as Critical
#  - HIGH SEVERITY: Count of all severities marked as High
#  - MEDIUM SEVERITY: Count of all severities marked as Medium
#  - LOW SEVERITY: Count of all severities marked as Low
#  - UNKNOWN SEVERITY: Count of all severities marked as Unknown
def create_osv_sca_summary_comparison_report(osv_df, sca_df):
    new_df = sca_df.copy()
    for index, row in new_df.iterrows():
        # Get values from SCA tool dataframe
        source_project           = new_df.loc[index, 'Project']
        source_total_cve         = new_df.loc[index, 'Total CVE']
        source_unique_cve        = new_df.loc[index, 'Unique CVE']
        source_critical_severity = new_df.loc[index, 'Critical Severity']
        source_high_severity     = new_df.loc[index, 'High Severity']
        source_medium_severity   = new_df.loc[index, 'Medium Severity']
        source_low_severity      = new_df.loc[index, 'Low Severity']
        source_unknown_severity  = new_df.loc[index, 'Unknown Severity']

        # Get rows from OSV dataframe that match project name
        df_osv_project           = osv_df.loc[osv_df['Project'].str.upper() == source_project.upper()].copy()

        # Get values from OSV dataframe
        osv_total_cve            = df_osv_project['Total CVE'].iloc[0]
        osv_unique_cve           = df_osv_project['Unique CVE'].iloc[0]
        osv_critical_severity    = df_osv_project['Critical Severity'].iloc[0]
        osv_high_severity        = df_osv_project['High Severity'].iloc[0]
        osv_medium_severity      = df_osv_project['Medium Severity'].iloc[0]
        osv_low_severity         = df_osv_project['Low Severity'].iloc[0]
        osv_unknown_severity     = df_osv_project['Unknown Severity'].iloc[0]

        # If any values match, prepend with "match_"
        # MatPlotLib table will change to green colour to represent matching values
        if str(source_total_cve) == str(osv_total_cve):
            new_df.loc[index, 'Total CVE'] = f'match_{source_total_cve}'
        if str(source_unique_cve) == str(osv_unique_cve):
            new_df.loc[index, 'Unique CVE'] = f'match_{source_unique_cve}'
        if str(source_critical_severity) == str(osv_critical_severity):
            new_df.loc[index, 'Critical Severity'] = f'match_{source_critical_severity}'
        if str(source_high_severity) == str(osv_high_severity):
            new_df.loc[index, 'High Severity'] = f'match_{source_high_severity}'
        if str(source_medium_severity) == str(osv_medium_severity):
            new_df.loc[index, 'Medium Severity'] = f'match_{source_medium_severity}'
        if str(source_low_severity) == str(osv_low_severity):
            new_df.loc[index, 'Low Severity'] = f'match_{source_low_severity}'
        if str(source_unknown_severity) == str(osv_unknown_severity):
            new_df.loc[index, 'Unknown Severity'] = f'match_{source_unknown_severity}'

    return new_df

In [ ]:
# ==================
# SAST Summary Report
# ==================
#
# Creates a SAST tool summary report containing the following columns:
#  - PROJECT: Experiment project name
#  - TAG: Experiment tag name
#  - TOTAL CWE: Count of all CWE IDs reported by SAST tool
#  - UNIQUE CWE: Count of unique CWE IDs reported by SAST tool
#  - VERY-HIGH CONFIDENCE: Count of severity confidence marked as Very-High
#  - HIGH CONFIDENCE: Count of severity confidence marked as High
#  - MEDIUM CONFIDENCE: Count of severity confidence marked as Medium
#  - LOW CONFIDENCE: Count of severity confidence marked as Low
#  - UNKNOWN CONFIDENCE: Count of severity confidence marked as Unknown
#  - CRITICAL SEVERITY: Count of all severities marked as Critical
#  - HIGH SEVERITY: Count of all severities marked as High
#  - MEDIUM SEVERITY: Count of all severities marked as Medium
#  - LOW SEVERITY: Count of all severities marked as Low
#  - UNKNOWN SEVERITY: Count of all severities marked as Unknown
#  - OWASP TOP 10: Count of all CWE IDs within the OWASP Top 10
#  - MITRE TOP 25: Count of all CWE IDs within the MITRE Top 25
def create_sast_summary_report(source_df, operating_system):
    # Remove rows that are not in main source code
    source_df = source_df.loc[source_df[get_main_source_code()].str.upper() == 'YES']

    if operating_system is not None:
        # Get rows matching operating system
        source_df = source_df.loc[source_df[get_experiment_runner_operating_system()].str.upper() == operating_system.upper()]
        new_df    = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_experiment_runner_operating_system(), get_cwe_id(), get_severity(), get_confidence(), get_owasp_top_ten(), get_mitre_top_twenty_five()]].copy()
    else:
        new_df = source_df[[get_experiment_project_name(), get_experiment_github_branch(), get_cwe_id(), get_severity(), get_confidence(), get_owasp_top_ten(), get_mitre_top_twenty_five()]].copy()

    # Create new columns with calculated data
    total_cwe_count            = new_df.groupby(get_experiment_project_name())[get_cwe_id()].transform('count')
    unique_cwe_count           = new_df.groupby(get_experiment_project_name())[get_cwe_id()].transform('nunique').astype(int).astype(str)

    very_high_confidence_count = new_df.groupby(get_experiment_project_name())[get_confidence()].transform(lambda x: x.value_counts().loc[x.unique()].get('Very-High', 0))
    high_confidence_count      = new_df.groupby(get_experiment_project_name())[get_confidence()].transform(lambda x: x.value_counts().loc[x.unique()].get('High', 0))
    medium_confidence_count    = new_df.groupby(get_experiment_project_name())[get_confidence()].transform(lambda x: x.value_counts().loc[x.unique()].get('Medium', 0))
    low_confidence_count       = new_df.groupby(get_experiment_project_name())[get_confidence()].transform(lambda x: x.value_counts().loc[x.unique()].get('Low', 0))
    unknown_confidence_count   = new_df.groupby(get_experiment_project_name())[get_confidence()].transform(lambda x: x.value_counts().loc[x.unique()].get('Unknown', 0))

    critical_severity_count = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('Critical', 0))
    high_severity_count     = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('High', 0))
    medium_severity_count   = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('Medium', 0))
    low_severity_count      = new_df.groupby(get_experiment_project_name())[get_severity()].transform(lambda x: x.value_counts().loc[x.unique()].get('Low', 0))

    temp_owasp_df      = new_df.loc[new_df[get_owasp_top_ten()].str.startswith('A')]
    owasp_top_10_count = temp_owasp_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')

    temp_mitre_df      = new_df.loc[~new_df[get_mitre_top_twenty_five()].str.startswith('N')]
    mitre_top_25_count = temp_mitre_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')

    if operating_system is None:
        # Ignore duplicate results per OS
        total_cwe_count            /= 3

        very_high_confidence_count /= 3
        high_confidence_count      /= 3
        medium_confidence_count    /= 3
        low_confidence_count       /= 3
        unknown_confidence_count   /= 3

        critical_severity_count    /= 3
        high_severity_count        /= 3
        medium_severity_count      /= 3
        low_severity_count         /= 3

        owasp_top_10_count         /= 3
        mitre_top_25_count         /= 3

    new_df['Total CWE']  = total_cwe_count
    new_df['Unique CWE'] = unique_cwe_count

    new_df['Very-High Confidence'] = very_high_confidence_count
    new_df['High Confidence']      = high_confidence_count
    new_df['Medium Confidence']    = medium_confidence_count
    new_df['Low Confidence']       = low_confidence_count
    new_df['Unknown Confidence']   = unknown_confidence_count

    new_df['Critical Severity'] = critical_severity_count
    new_df['High Severity']     = high_severity_count
    new_df['Medium Severity']   = medium_severity_count
    new_df['Low Severity']      = low_severity_count

    new_df['Total OWASP Top 10'] = owasp_top_10_count
    new_df['Total OWASP Top 10'] = new_df['Total OWASP Top 10'].fillna(0)

    new_df['Total MITRE Top 25'] = mitre_top_25_count
    new_df['Total MITRE Top 25'] = new_df['Total MITRE Top 25'].fillna(0)

    # Convert from float -> int -> string
    new_df['Total CWE']            = new_df['Total CWE'].astype(int).astype(str)
    new_df['Very-High Confidence'] = new_df['Very-High Confidence'].astype(int).astype(str)
    new_df['High Confidence']      = new_df['High Confidence'].astype(int).astype(str)
    new_df['Medium Confidence']    = new_df['Medium Confidence'].astype(int).astype(str)
    new_df['Low Confidence']       = new_df['Low Confidence'].astype(int).astype(str)
    new_df['Unknown Confidence']   = new_df['Unknown Confidence'].astype(int).astype(str)
    new_df['Critical Severity']    = new_df['Critical Severity'].astype(int).astype(str)
    new_df['High Severity']        = new_df['High Severity'].astype(int).astype(str)
    new_df['Medium Severity']      = new_df['Medium Severity'].astype(int).astype(str)
    new_df['Low Severity']         = new_df['Low Severity'].astype(int).astype(str)
    new_df['Total OWASP Top 10']   = new_df['Total OWASP Top 10'].astype(int).astype(str)
    new_df['Total MITRE Top 25']   = new_df['Total MITRE Top 25'].astype(int).astype(str)

    # Remove unused columns
    new_df = new_df.drop(get_cwe_id(), axis=1)
    new_df = new_df.drop(get_severity(), axis=1)
    new_df = new_df.drop(get_confidence(), axis=1)
    new_df = new_df.drop(get_owasp_top_ten(), axis=1)
    new_df = new_df.drop(get_mitre_top_twenty_five(), axis=1)

    # Drop duplicate values
    new_df = new_df.drop_duplicates(ignore_index=True)

    # Set column names
    new_df = new_df.rename(
        columns = {
            get_experiment_project_name() : 'Project',
            get_experiment_github_branch() : 'Tag',
            get_experiment_runner_operating_system() : 'Runner OS'
        }
    )

    # Sort column by Project and MITRE Top 25
    new_df = new_df.sort_values(['Project', 'Total MITRE Top 25'])

    # Drop duplicates of projects but keep last
    new_df.drop_duplicates(subset='Project', keep='last', inplace=True)

    # Reset index
    new_df.reset_index(drop=True, inplace=True)

    return new_df

In [ ]:
# ========================
# SAST OWASP Top 10 Report
# ========================
#
# Creates a SAST tool summary report containing the following columns:
#  - PROJECT: Experiment project name
#  - TAG: Experiment tag name
#  - CWE ID: CWE identified
#  - OWASP TOP 10: Top 10 identified
def create_sast_owasp_report(source_df, operating_system):
    # Remove rows that are not in main source code
    source_df = source_df.loc[source_df[get_main_source_code()].str.upper() == 'YES']
    source_df = source_df.loc[source_df[get_owasp_top_ten()].str.upper() != 'NO']

    if operating_system is not None:
        # Get rows matching operating system
        source_df = source_df.loc[source_df[get_experiment_runner_operating_system()].str.upper() == operating_system.upper()]
        new_df    = source_df[[get_experiment_project_name(), get_experiment_runner_operating_system(), get_cwe_id(), get_owasp_top_ten()]].copy()
    else:
        new_df = source_df[[get_experiment_project_name(), get_cwe_id(), get_owasp_top_ten()]].copy()

    a01_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A01')]
    a02_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A02')]
    a03_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A03')]
    a04_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A04')]
    a05_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A05')]
    a06_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A06')]
    a07_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A07')]
    a08_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A08')]
    a09_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A09')]
    a10_df = source_df.loc[source_df[get_owasp_top_ten()].astype(str).str.startswith('A10')]

    a01_count = a01_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a02_count = a02_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a03_count = a03_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a04_count = a04_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a05_count = a05_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a06_count = a06_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a07_count = a07_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a08_count = a08_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a09_count = a09_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')
    a10_count = a10_df.groupby(get_experiment_project_name())[get_owasp_top_ten()].transform('count')

    if operating_system is None:
        # Ignore duplicate results per OS
        a01_count /= 3
        a02_count /= 3
        a03_count /= 3
        a04_count /= 3
        a05_count /= 3
        a06_count /= 3
        a07_count /= 3
        a08_count /= 3
        a09_count /= 3
        a10_count /= 3

    new_df['A01'] = a01_count if str(a01_count) is not None else 0
    new_df['A02'] = a02_count if str(a02_count) is not None else 0
    new_df['A03'] = a03_count if str(a03_count) is not None else 0
    new_df['A04'] = a04_count if str(a04_count) is not None else 0
    new_df['A05'] = a05_count if str(a05_count) is not None else 0
    new_df['A06'] = a06_count if str(a06_count) is not None else 0
    new_df['A07'] = a07_count if str(a07_count) is not None else 0
    new_df['A08'] = a08_count if str(a08_count) is not None else 0
    new_df['A09'] = a09_count if str(a09_count) is not None else 0
    new_df['A10'] = a10_count if str(a10_count) is not None else 0

    # Remove unused columns
    new_df = new_df.drop(get_cwe_id(), axis=1)
    new_df = new_df.drop(get_owasp_top_ten(), axis=1)

    # Merge rows for project
    if operating_system is not None:
      new_df = new_df.groupby([get_experiment_project_name(), get_experiment_runner_operating_system()])[['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10']].first().reset_index()
    else:
      new_df = new_df.groupby(get_experiment_project_name())[['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10']].first().reset_index()

    # Drop duplicate values
    new_df = new_df.drop_duplicates(ignore_index=True)

    # Set column names
    new_df = new_df.rename(
        columns = {
            get_experiment_project_name() : 'Project',
            get_experiment_github_branch() : 'Tag',
            get_experiment_runner_operating_system() : 'Runner OS'
        }
    )

    # Sort column by Project
    new_df = new_df.sort_values('Project')

    # Reset index
    new_df.reset_index(drop=True, inplace=True)

    return new_df

In [ ]:
# ========================
# SAST MITRE Top 25 Report
# ========================
#
# Creates a SAST tool summary report containing the following columns:
#  - PROJECT: Experiment project name
#  - TAG: Experiment tag name
#  - CWE ID: CWE identified
#  - MITRE Top 25: Top 25 identified
def create_sast_mitre_report(source_df, operating_system):
    # Remove rows that are not in main source code
    source_df = source_df.loc[source_df[get_main_source_code()].str.upper() == 'YES']
    source_df = source_df.loc[source_df[get_mitre_top_twenty_five()].str.upper() != 'NO']

    if operating_system is not None:
        # Get rows matching operating system
        source_df = source_df.loc[source_df[get_experiment_runner_operating_system()].str.upper() == operating_system.upper()]
        new_df    = source_df[[get_experiment_project_name(), get_experiment_runner_operating_system(), get_cwe_id(), get_mitre_top_twenty_five()]].copy()
    else:
        new_df = source_df[[get_experiment_project_name(), get_cwe_id(), get_mitre_top_twenty_five()]].copy()

    m01_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('1')]
    m02_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('2')]
    m03_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('3')]
    m04_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('4')]
    m05_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('5')]
    m06_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('6')]
    m07_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('7')]
    m08_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('8')]
    m09_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('9')]
    m10_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('10')]
    m11_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('11')]
    m12_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('12')]
    m13_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('13')]
    m14_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('14')]
    m15_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('15')]
    m16_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('16')]
    m17_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('17')]
    m18_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('18')]
    m19_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('19')]
    m20_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('20')]
    m21_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('21')]
    m22_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('22')]
    m23_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('23')]
    m24_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('24')]
    m25_df = source_df.loc[source_df[get_mitre_top_twenty_five()].astype(str).str.fullmatch('25')]

    m01_count = m01_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m02_count = m02_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m03_count = m03_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m04_count = m04_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m05_count = m05_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m06_count = m06_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m07_count = m07_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m08_count = m08_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m09_count = m09_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m10_count = m10_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m11_count = m11_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m12_count = m12_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m13_count = m13_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m14_count = m14_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m15_count = m15_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m16_count = m16_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m17_count = m17_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m18_count = m18_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m19_count = m19_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m20_count = m20_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m21_count = m21_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m22_count = m22_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m23_count = m23_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m24_count = m24_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')
    m25_count = m25_df.groupby(get_experiment_project_name())[get_mitre_top_twenty_five()].transform('count')

    if operating_system is None:
        # Ignore duplicate results per OS
        m01_count /= 3
        m02_count /= 3
        m03_count /= 3
        m04_count /= 3
        m05_count /= 3
        m06_count /= 3
        m07_count /= 3
        m08_count /= 3
        m09_count /= 3
        m10_count /= 3
        m11_count /= 3
        m12_count /= 3
        m13_count /= 3
        m14_count /= 3
        m15_count /= 3
        m16_count /= 3
        m17_count /= 3
        m18_count /= 3
        m19_count /= 3
        m20_count /= 3
        m21_count /= 3
        m22_count /= 3
        m23_count /= 3
        m24_count /= 3
        m25_count /= 3

    new_df['M01'] = m01_count if str(m01_count) is not None else 0
    new_df['M02'] = m02_count if str(m02_count) is not None else 0
    new_df['M03'] = m03_count if str(m03_count) is not None else 0
    new_df['M04'] = m04_count if str(m04_count) is not None else 0
    new_df['M05'] = m05_count if str(m05_count) is not None else 0
    new_df['M06'] = m06_count if str(m06_count) is not None else 0
    new_df['M07'] = m07_count if str(m07_count) is not None else 0
    new_df['M08'] = m08_count if str(m08_count) is not None else 0
    new_df['M09'] = m09_count if str(m09_count) is not None else 0
    new_df['M10'] = m10_count if str(m10_count) is not None else 0
    new_df['M11'] = m11_count if str(m11_count) is not None else 0
    new_df['M12'] = m12_count if str(m12_count) is not None else 0
    new_df['M13'] = m13_count if str(m13_count) is not None else 0
    new_df['M14'] = m14_count if str(m14_count) is not None else 0
    new_df['M15'] = m15_count if str(m15_count) is not None else 0
    new_df['M16'] = m16_count if str(m16_count) is not None else 0
    new_df['M17'] = m17_count if str(m17_count) is not None else 0
    new_df['M18'] = m18_count if str(m18_count) is not None else 0
    new_df['M19'] = m19_count if str(m19_count) is not None else 0
    new_df['M20'] = m20_count if str(m20_count) is not None else 0
    new_df['M21'] = m21_count if str(m21_count) is not None else 0
    new_df['M22'] = m22_count if str(m22_count) is not None else 0
    new_df['M23'] = m23_count if str(m23_count) is not None else 0
    new_df['M24'] = m24_count if str(m24_count) is not None else 0
    new_df['M25'] = m25_count if str(m25_count) is not None else 0

    # Remove unused columns
    new_df = new_df.drop(get_cwe_id(), axis=1)
    new_df = new_df.drop(get_mitre_top_twenty_five(), axis=1)

    # Merge rows for project
    if operating_system is not None:
      new_df = new_df.groupby([get_experiment_project_name(), get_experiment_runner_operating_system()])[['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20', 'M21', 'M22', 'M23', 'M24', 'M25']].first().reset_index()
    else:
      new_df = new_df.groupby(get_experiment_project_name())[['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20', 'M21', 'M22', 'M23', 'M24', 'M25']].first().reset_index()

    # Drop duplicate values
    new_df = new_df.drop_duplicates(ignore_index=True)

    # Set column names
    new_df = new_df.rename(
        columns = {
            get_experiment_project_name() : 'Project',
            get_experiment_github_branch() : 'Tag',
            get_experiment_runner_operating_system() : 'Runner OS'
        }
    )

    # Sort column by Project
    new_df = new_df.sort_values('Project')

    # Reset index
    new_df.reset_index(drop=True, inplace=True)

    return new_df

<hr style="border:2px solid gray">

## Open Source Vulnerability Reports

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Remove rows that do not contain CVE IDs
df_cyclonedx_sbom_osv_cve = df_cyclonedx_sbom_osv_report.loc[df_cyclonedx_sbom_osv_report[get_osv_vulnerability_cve()].str.upper() != 'NOT FOUND']

# Create dataframe of OSV summary report
df_osv_summary = create_osv_summary_report(df_cyclonedx_sbom_osv_cve)

# Write data to excel report
save_excel_sheet(df_osv_summary, 'osv_summary')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Remove unused columns
df_osv_summary_cve = df_osv_summary.drop(['Required Dependency','Optional Dependency'], axis=1)

# Create MatPlotLib table
osv_cve_summary_table = render_matplotlib_table(df_osv_summary_cve, "Open-Source Vulnerability - CVE Summary")

# Add separator between experiment groups
set_row_edge_color(osv_cve_summary_table, 5,  'k')
set_row_edge_color(osv_cve_summary_table, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/open_source_vulnerability/osv_cve_summary.png', bbox_inches='tight')
plt.close()

<hr style="border:2px solid gray">

## Software Composition Analysis Reports

### Grype

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of SCA Grype summary report
df_grype_macos  = create_sca_summary_report(df_grype_data, get_os_macos())
df_grype_ubuntu = create_sca_summary_report(df_grype_data, get_os_ubuntu())

# Compare results with Open Source Vulnerability report
df_grype_macos_compared  = create_osv_sca_summary_comparison_report(df_osv_summary, df_grype_macos)
df_grype_ubuntu_compared = create_osv_sca_summary_comparison_report(df_osv_summary, df_grype_ubuntu)

# Combine all operating system reports
df_grype_summary          = pd.concat([df_grype_macos, df_grype_ubuntu], ignore_index=True, sort=False)
df_grype_summary_compared = pd.concat([df_grype_macos_compared, df_grype_ubuntu_compared], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_grype_summary, f'{get_grype()}_summary')
save_excel_sheet(df_grype_summary_compared, f'{get_grype()}_summary_compared')

# ===========================
# DIRECT & TRANSITIVE REPORTS
# ===========================

# Write direct and transitive reports to excel report
create_sca_dependencies_reports(df_grype_data, get_os_macos(), get_grype(), True)
create_sca_dependencies_reports(df_grype_data, get_os_ubuntu(), get_grype(), False)

# ========================================
# MATCHING CVE REPORT - TRUE POSITIVE DATA
# ========================================

# Create dataframe of CVE ID data that matches OSV data
df_grype_cve_macos_tp  = create_sca_osv_cve_comparison_report(df_grype_data, df_cyclonedx_sbom_osv_cve, get_os_macos())
df_grype_cve_ubuntu_tp = create_sca_osv_cve_comparison_report(df_grype_data, df_cyclonedx_sbom_osv_cve, get_os_ubuntu())

# Combine all operating system reports
df_grype_cve_tp = pd.concat([df_grype_cve_macos_tp, df_grype_cve_ubuntu_tp], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_grype_cve_tp, f'{get_grype()}_tp_comparison')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
grype_macos_title  = f'Software Composition Analysis - {get_grype().capitalize()} - CVE Summary ({get_os_macos()})'
grype_ubuntu_title = f'Software Composition Analysis - {get_grype().capitalize()} - CVE Summary ({get_os_ubuntu()})'

# ===== MACOS =====
grype_summary_table_macos = render_matplotlib_table(df_grype_macos_compared, grype_macos_title)

# Add separator between experiment groups
set_row_edge_color(grype_summary_table_macos, 5,  'k')
set_row_edge_color(grype_summary_table_macos, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_grype()}/{get_grype()}_summary_macos.png', bbox_inches='tight')
plt.close()

# ===== UBUNTU =====
grype_summary_table_ubuntu = render_matplotlib_table(df_grype_ubuntu_compared, grype_ubuntu_title)

# Add separator between experiment groups
set_row_edge_color(grype_summary_table_ubuntu, 5,  'k')
set_row_edge_color(grype_summary_table_ubuntu, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_grype()}/{get_grype()}_summary_ubuntu.png', bbox_inches='tight')
plt.close()

# =============================
# MATPLOT TABLE - OS COMPARISON
# =============================

# Create dataframe for total CVE per operating system
df_grype_os_comparison = df_grype_macos.filter(['Project','Tag'], axis=1)
df_grype_os_comparison[f'Total CVE - {get_os_macos()}']  = df_grype_macos.filter(['Total CVE'], axis=1)
df_grype_os_comparison[f'Total CVE - {get_os_ubuntu()}'] = df_grype_ubuntu.filter(['Total CVE'], axis=1)

# Set chart titles
grype_comparison_title = f'Software Composition Analysis - {get_grype().capitalize()} - Runner OS Comparison'

# Create MatPlotLib table
df_grype_os_comparison_table = render_matplotlib_table(df_grype_os_comparison, grype_comparison_title)

# # Add separator between experiment groups
set_row_edge_color(df_grype_os_comparison_table, 5,  'k')
set_row_edge_color(df_grype_os_comparison_table, 10, 'k')

# # Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_grype()}/{get_grype()}_os_comparison.png', bbox_inches='tight')
plt.close()

### Snyk

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of SCA Snyk summary report
df_snyk_macos   = create_sca_summary_report(df_snyk_data, get_os_macos())
df_snyk_ubuntu  = create_sca_summary_report(df_snyk_data, get_os_ubuntu())
df_snyk_windows = create_sca_summary_report(df_snyk_data, get_os_windows())

# Compare results with Open Source Vulnerability report
df_snyk_macos_compared   = create_osv_sca_summary_comparison_report(df_osv_summary, df_snyk_macos)
df_snyk_ubuntu_compared  = create_osv_sca_summary_comparison_report(df_osv_summary, df_snyk_ubuntu)
df_snyk_windows_compared = create_osv_sca_summary_comparison_report(df_osv_summary, df_snyk_windows)

# Combine all operating system reports
df_snyk_summary          = pd.concat([df_snyk_macos, df_snyk_ubuntu, df_snyk_windows], ignore_index=True, sort=False)
df_snyk_summary_compared = pd.concat([df_snyk_macos_compared, df_snyk_ubuntu_compared, df_snyk_windows_compared], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_snyk_summary, f'{get_snyk()}_summary')
save_excel_sheet(df_snyk_summary_compared, f'{get_snyk()}_summary_compared')

# ===========================
# DIRECT & TRANSITIVE REPORTS
# ===========================

# Write direct and transitive reports to excel report
create_sca_dependencies_reports(df_snyk_data, get_os_macos(), get_snyk(), True)
create_sca_dependencies_reports(df_snyk_data, get_os_ubuntu(), get_snyk(), False)
create_sca_dependencies_reports(df_snyk_data, get_os_windows(), get_snyk(), False)

# ========================================
# MATCHING CVE REPORT - TRUE POSITIVE DATA
# ========================================

# Create dataframe of CVE ID data that matches OSV data
df_snyk_cve_macos_tp   = create_sca_osv_cve_comparison_report(df_snyk_data, df_cyclonedx_sbom_osv_cve, get_os_macos())
df_snyk_cve_ubuntu_tp  = create_sca_osv_cve_comparison_report(df_snyk_data, df_cyclonedx_sbom_osv_cve, get_os_ubuntu())
df_snyk_cve_windows_tp = create_sca_osv_cve_comparison_report(df_snyk_data, df_cyclonedx_sbom_osv_cve, get_os_windows())

# Combine all operating system reports
df_snyk_cve_tp = pd.concat([df_snyk_cve_macos_tp, df_snyk_cve_ubuntu_tp, df_snyk_cve_windows_tp], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_snyk_cve_tp, f'{get_snyk()}_tp_comparison')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
snyk_macos_title   = f'Software Composition Analysis - {get_snyk().capitalize()} - CVE Summary ({get_os_macos()})'
snyk_ubuntu_title  = f'Software Composition Analysis - {get_snyk().capitalize()} - CVE Summary ({get_os_ubuntu()})'
snyk_windows_title = f'Software Composition Analysis - {get_snyk().capitalize()} - CVE Summary ({get_os_windows()})'

# ===== MACOS =====
snyk_summary_table_macos = render_matplotlib_table(df_snyk_macos_compared, snyk_macos_title)

# Add separator between experiment groups
set_row_edge_color(snyk_summary_table_macos, 5,  'k')
set_row_edge_color(snyk_summary_table_macos, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_snyk()}/{get_snyk()}_summary_macos.png', bbox_inches='tight')
plt.close()

# ===== UBUNTU =====
snyk_summary_table_ubuntu = render_matplotlib_table(df_snyk_ubuntu_compared, snyk_ubuntu_title)

# Add separator between experiment groups
set_row_edge_color(snyk_summary_table_ubuntu, 5,  'k')
set_row_edge_color(snyk_summary_table_ubuntu, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_snyk()}/{get_snyk()}_summary_ubuntu.png', bbox_inches='tight')
plt.close()

# ===== WINDOWS =====
snyk_summary_table_windows = render_matplotlib_table(df_snyk_windows_compared, snyk_windows_title)

# Add separator between experiment groups
set_row_edge_color(snyk_summary_table_windows, 5,  'k')
set_row_edge_color(snyk_summary_table_windows, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_snyk()}/{get_snyk()}_summary_windows.png', bbox_inches='tight')
plt.close()

# =============================
# MATPLOT TABLE - OS COMPARISON
# =============================

# Create dataframe for total CVE per operating system
df_snyk_os_comparison = df_snyk_macos.filter(['Project','Tag'], axis=1)
df_snyk_os_comparison[f'Total CVE - {get_os_macos()}']  = df_snyk_macos.filter(['Total CVE'], axis=1)
df_snyk_os_comparison[f'Total CVE - {get_os_ubuntu()}'] = df_snyk_ubuntu.filter(['Total CVE'], axis=1)
df_snyk_os_comparison[f'Total CVE - {get_os_windows()}'] = df_snyk_windows.filter(['Total CVE'], axis=1)

# Set chart titles
snyk_comparison_title  = f'Software Composition Analysis - {get_snyk().capitalize()} - Runner OS Comparison'

# Create MatPlotLib table
df_snyk_os_comparison_table = render_matplotlib_table(df_snyk_os_comparison, snyk_comparison_title)

# # Add separator between experiment groups
set_row_edge_color(df_snyk_os_comparison_table, 5,  'k')
set_row_edge_color(df_snyk_os_comparison_table, 10, 'k')

# # Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_snyk()}/{get_snyk()}_os_comparison.png', bbox_inches='tight')
plt.close()

### OWASP Dependency Check


In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of SCA OWASP Dependency Check summary report
df_owasp_dependency_check_macos   = create_sca_summary_report(df_owasp_dependency_check_data, get_os_macos())
df_owasp_dependency_check_ubuntu  = create_sca_summary_report(df_owasp_dependency_check_data, get_os_ubuntu())
df_owasp_dependency_check_windows = create_sca_summary_report(df_owasp_dependency_check_data, get_os_windows())

# Compare results with Open Source Vulnerability report
df_owasp_dependency_check_macos_compared   = create_osv_sca_summary_comparison_report(df_osv_summary, df_owasp_dependency_check_macos)
df_owasp_dependency_check_ubuntu_compared  = create_osv_sca_summary_comparison_report(df_osv_summary, df_owasp_dependency_check_ubuntu)
df_owasp_dependency_check_windows_compared = create_osv_sca_summary_comparison_report(df_osv_summary, df_owasp_dependency_check_windows)

# Combine all operating system reports
df_owasp_dependency_check_summary          = pd.concat([df_owasp_dependency_check_macos, df_owasp_dependency_check_ubuntu, df_owasp_dependency_check_windows], ignore_index=True, sort=False)
df_owasp_dependency_check_summary_compared = pd.concat([df_owasp_dependency_check_macos_compared, df_owasp_dependency_check_ubuntu_compared, df_owasp_dependency_check_windows_compared], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_owasp_dependency_check_summary, f'{get_owasp_dependency_check_alias()}_summary')
save_excel_sheet(df_owasp_dependency_check_summary_compared, f'{get_owasp_dependency_check_alias()}_compared')

# ===========================
# DIRECT & TRANSITIVE REPORTS
# ===========================

# Write direct and transitive reports to excel report
create_sca_dependencies_reports(df_owasp_dependency_check_data, get_os_macos(), get_owasp_dependency_check_alias(), True)
create_sca_dependencies_reports(df_owasp_dependency_check_data, get_os_ubuntu(), get_owasp_dependency_check_alias(), False)
create_sca_dependencies_reports(df_owasp_dependency_check_data, get_os_windows(), get_owasp_dependency_check_alias(), False)

# ========================================
# MATCHING CVE REPORT - TRUE POSITIVE DATA
# ========================================

# Create dataframe of CVE ID data that matches OSV data
df_owasp_dependency_check_cve_macos_tp   = create_sca_osv_cve_comparison_report(df_owasp_dependency_check_data, df_cyclonedx_sbom_osv_cve, get_os_macos())
df_owasp_dependency_check_cve_ubuntu_tp  = create_sca_osv_cve_comparison_report(df_owasp_dependency_check_data, df_cyclonedx_sbom_osv_cve, get_os_ubuntu())
df_owasp_dependency_check_cve_windows_tp = create_sca_osv_cve_comparison_report(df_owasp_dependency_check_data, df_cyclonedx_sbom_osv_cve, get_os_windows())

# Combine all operating system reports
df_owasp_dependency_check_cve_tp = pd.concat([df_owasp_dependency_check_cve_macos_tp, df_owasp_dependency_check_cve_ubuntu_tp, df_owasp_dependency_check_cve_windows_tp], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_owasp_dependency_check_cve_tp, f'{get_owasp_dependency_check_alias()}_tp_comparison')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
owasp_dependency_check_macos_title   = f'Software Composition Analysis - {get_owasp_dependency_check()} - CVE Summary ({get_os_macos()})'
owasp_dependency_check_ubuntu_title  = f'Software Composition Analysis - {get_owasp_dependency_check()} - CVE Summary ({get_os_ubuntu()})'
owasp_dependency_check_windows_title = f'Software Composition Analysis - {get_owasp_dependency_check()} - CVE Summary ({get_os_windows()})'

# ===== MACOS =====
owasp_dependency_check_summary_table_macos = render_matplotlib_table(df_owasp_dependency_check_macos_compared, owasp_dependency_check_macos_title)

# Add separator between experiment groups
set_row_edge_color(owasp_dependency_check_summary_table_macos, 5,  'k')
set_row_edge_color(owasp_dependency_check_summary_table_macos, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_owasp_dependency_check_alias()}/{get_owasp_dependency_check_alias()}_summary_macos.png', bbox_inches='tight')
plt.close()

# ===== UBUNTU =====
owasp_dependency_check_summary_table_ubuntu = render_matplotlib_table(df_owasp_dependency_check_ubuntu_compared, owasp_dependency_check_ubuntu_title)

# Add separator between experiment groups
set_row_edge_color(owasp_dependency_check_summary_table_ubuntu, 5,  'k')
set_row_edge_color(owasp_dependency_check_summary_table_ubuntu, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_owasp_dependency_check_alias()}/{get_owasp_dependency_check_alias()}_summary_ubuntu.png', bbox_inches='tight')
plt.close()

# ===== WINDOWS =====
owasp_dependency_check_summary_table_windows = render_matplotlib_table(df_owasp_dependency_check_windows_compared, owasp_dependency_check_windows_title)

# Add separator between experiment groups
set_row_edge_color(owasp_dependency_check_summary_table_windows, 5,  'k')
set_row_edge_color(owasp_dependency_check_summary_table_windows, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_owasp_dependency_check_alias()}/{get_owasp_dependency_check_alias()}_summary_windows.png', bbox_inches='tight')
plt.close()

# =============================
# MATPLOT TABLE - OS COMPARISON
# =============================

# Create dataframe for total CVE per operating system
df_owasp_dependency_check_os_comparison = df_owasp_dependency_check_macos.filter(['Project','Tag'], axis=1)
df_owasp_dependency_check_os_comparison[f'Total CVE - {get_os_macos()}']  = df_owasp_dependency_check_macos.filter(['Total CVE'], axis=1)
df_owasp_dependency_check_os_comparison[f'Total CVE - {get_os_ubuntu()}'] = df_owasp_dependency_check_ubuntu.filter(['Total CVE'], axis=1)
df_owasp_dependency_check_os_comparison[f'Total CVE - {get_os_windows()}'] = df_owasp_dependency_check_windows.filter(['Total CVE'], axis=1)

# Set chart titles
owasp_dependency_check_comparison_title  = f'Software Composition Analysis - {get_owasp_dependency_check()} - Runner OS Comparison'

# Create MatPlotLib table
df_owasp_dependency_check_os_comparison_table = render_matplotlib_table(df_owasp_dependency_check_os_comparison, owasp_dependency_check_comparison_title)

# # Add separator between experiment groups
set_row_edge_color(df_owasp_dependency_check_os_comparison_table, 5,  'k')
set_row_edge_color(df_owasp_dependency_check_os_comparison_table, 10, 'k')

# # Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_owasp_dependency_check_alias()}/{get_owasp_dependency_check_alias()}_os_comparison.png', bbox_inches='tight')
plt.close()

### Dependabot

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of Dependabot Snyk summary report
df_dependabot_summary = create_sca_summary_report(df_dependabot_data, None)

# Compare results with Open Source Vulnerability report
df_dependabot_summary_compared = create_osv_sca_summary_comparison_report(df_osv_summary, df_dependabot_summary)

# Write data to excel report
save_excel_sheet(df_dependabot_summary, f'{get_dependabot()}_summary')
save_excel_sheet(df_dependabot_summary_compared, f'{get_dependabot()}_summary_compared')

# ===========================
# DIRECT & TRANSITIVE REPORTS
# ===========================

# Write direct and transitive reports to excel report
create_sca_dependencies_reports(df_dependabot_data, None, get_dependabot(), True)

# ========================================
# MATCHING CVE REPORT - TRUE POSITIVE DATA
# ========================================

# Create dataframe of CVE ID data that matches OSV data
df_dependabot_cve_tp = create_sca_osv_cve_comparison_report(df_dependabot_data, df_cyclonedx_sbom_osv_cve, None)

# Write data to excel report
save_excel_sheet(df_dependabot_cve_tp, f'{get_dependabot()}_tp_comparison')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
dependabot_title = f'Software Composition Analysis - {get_dependabot().capitalize()} - CVE Summary'

dependabot_summary_table = render_matplotlib_table(df_dependabot_summary_compared, dependabot_title)

# Add separator between experiment groups
set_row_edge_color(dependabot_summary_table, 5,  'k')
set_row_edge_color(dependabot_summary_table, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_dependabot()}/{get_dependabot()}_summary.png', bbox_inches='tight')
plt.close()

### Eclipse Steady

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of Eclipse Steady Snyk summary report
df_eclipse_steady_summary = create_sca_summary_report(df_eclipse_steady_data, get_os_ubuntu())

# Compare results with Open Source Vulnerability report
df_eclipse_steady_summary_compared = create_osv_sca_summary_comparison_report(df_osv_summary, df_eclipse_steady_summary)

# Write data to excel report
save_excel_sheet(df_eclipse_steady_summary, f'{get_eclipse_steady_alias()}_summary')
save_excel_sheet(df_eclipse_steady_summary_compared, f'{get_eclipse_steady_alias()}_summary_compared')

# ===========================
# DIRECT & TRANSITIVE REPORTS
# ===========================

# Write direct and transitive reports to excel report
create_sca_dependencies_reports(df_eclipse_steady_data, get_os_ubuntu(), get_eclipse_steady_alias(), True)

# ========================================
# MATCHING CVE REPORT - TRUE POSITIVE DATA
# ========================================

# Create dataframe of CVE ID data that matches OSV data
df_eclipse_steady_cve_tp = create_sca_osv_cve_comparison_report(df_eclipse_steady_data, df_cyclonedx_sbom_osv_cve, get_os_ubuntu())

# Write data to excel report
save_excel_sheet(df_eclipse_steady_cve_tp, f'{get_eclipse_steady_alias()}_tp_comparison')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
eclipse_steady_title = f'Software Composition Analysis - {get_eclipse_steady()} - CVE Summary ({get_os_ubuntu()})'

eclipse_steady_summary_table = render_matplotlib_table(df_eclipse_steady_summary_compared, eclipse_steady_title)

# Add separator between experiment groups
set_row_edge_color(eclipse_steady_summary_table, 5,  'k')
set_row_edge_color(eclipse_steady_summary_table, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/software_composition_analysis/{get_eclipse_steady_alias()}/{get_eclipse_steady_alias()}_summary.png', bbox_inches='tight')
plt.close()

<hr style="border:2px solid gray">

## Static Application Security Testing Reports

### CodeQL

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of SAST CodeQL summary report
df_codeql_macos   = create_sast_summary_report(df_codeql_data, get_os_macos())
df_codeql_ubuntu  = create_sast_summary_report(df_codeql_data, get_os_ubuntu())
df_codeql_windows = create_sast_summary_report(df_codeql_data, get_os_windows())

# Combine all operating system reports
df_codeql_summary = pd.concat([df_codeql_macos, df_codeql_ubuntu, df_codeql_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_codeql_summary, f'{get_codeql().lower()}_summary')

# ===================
# OWASP TOP 10 REPORT
# ===================

# Create dataframe of SAST CodeQL OWASP Top 10 report
df_codeql_owasp_macos   = create_sast_owasp_report(df_codeql_data, get_os_macos())
df_codeql_owasp_ubuntu  = create_sast_owasp_report(df_codeql_data, get_os_ubuntu())
df_codeql_owasp_windows = create_sast_owasp_report(df_codeql_data, get_os_windows())

# Combine all operating system reports
df_codeql_owasp = pd.concat([df_codeql_owasp_macos, df_codeql_owasp_ubuntu, df_codeql_owasp_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_codeql_owasp, f'{get_codeql().lower()}_owasp')

# ===================
# MITRE TOP 25 REPORT
# ===================

# Create dataframe of SAST CodeQL MITRE Top 25 report
df_codeql_mitre_macos   = create_sast_mitre_report(df_codeql_data, get_os_macos())
df_codeql_mitre_ubuntu  = create_sast_mitre_report(df_codeql_data, get_os_ubuntu())
df_codeql_mitre_windows = create_sast_mitre_report(df_codeql_data, get_os_windows())

# Combine all operating system reports
df_codeql_mitre = pd.concat([df_codeql_mitre_macos, df_codeql_mitre_ubuntu, df_codeql_mitre_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_codeql_mitre, f'{get_codeql().lower()}_mitre')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
codeql_macos_title   = f'Static Application Security Testing - {get_codeql()} - CWE Summary ({get_os_macos()})'
codeql_ubuntu_title  = f'Static Application Security Testing - {get_codeql()} - CWE Summary ({get_os_ubuntu()})'
codeql_windows_title = f'Static Application Security Testing - {get_codeql()} - CWE Summary ({get_os_windows()})'

# ===== MACOS =====
codeql_summary_table_macos = render_matplotlib_table(df_codeql_macos, codeql_macos_title)

# Add separator between experiment groups
set_row_edge_color(codeql_summary_table_macos, 3,  'k')
set_row_edge_color(codeql_summary_table_macos, 8, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_codeql().lower()}/{get_codeql().lower()}_summary_macos.png', bbox_inches='tight')
plt.close()

# ===== UBUNTU =====
codeql_summary_table_ubuntu = render_matplotlib_table(df_codeql_ubuntu, codeql_ubuntu_title)

# Add separator between experiment groups
set_row_edge_color(codeql_summary_table_ubuntu, 3,  'k')
set_row_edge_color(codeql_summary_table_ubuntu, 8, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_codeql().lower()}/{get_codeql().lower()}_summary_ubuntu.png', bbox_inches='tight')
plt.close()

# ===== WINDOWS =====
codeql_summary_table_windows = render_matplotlib_table(df_codeql_windows, codeql_windows_title)

# Add separator between experiment groups
set_row_edge_color(codeql_summary_table_windows, 3,  'k')
set_row_edge_color(codeql_summary_table_windows, 8, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_codeql().lower()}/{get_codeql().lower()}_summary_windows.png', bbox_inches='tight')
plt.close()

# =============================
# MATPLOT TABLE - OS COMPARISON
# =============================

# Create dataframe for total CWE per operating system
df_codeql_os_comparison = df_codeql_macos.filter(['Project','Tag'], axis=1)
df_codeql_os_comparison[f'Total CWE - {get_os_macos()}']   = df_codeql_macos.filter(['Total CWE'], axis=1)
df_codeql_os_comparison[f'Total CWE - {get_os_ubuntu()}']  = df_codeql_ubuntu.filter(['Total CWE'], axis=1)
df_codeql_os_comparison[f'Total CWE - {get_os_windows()}'] = df_codeql_windows.filter(['Total CWE'], axis=1)

# Drop duplicates but keep first
df_codeql_os_comparison.drop_duplicates(keep='first', inplace=True)

# Set chart titles
codeql_comparison_title = f'Static Application Security Testing - {get_codeql()} - Runner OS Comparison'

# Create MatPlotLib table
df_codeql_os_comparison_table = render_matplotlib_table(df_codeql_os_comparison, codeql_comparison_title)

# # Add separator between experiment groups
set_row_edge_color(df_codeql_os_comparison_table, 3,  'k')
set_row_edge_color(df_codeql_os_comparison_table, 8, 'k')

# # Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_codeql().lower()}/{get_codeql().lower()}_os_comparison.png', bbox_inches='tight')
plt.close()

### Horusec

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of SAST Horusec summary report
df_horusec_macos   = create_sast_summary_report(df_horusec_data, get_os_macos())
df_horusec_ubuntu  = create_sast_summary_report(df_horusec_data, get_os_ubuntu())
df_horusec_windows = create_sast_summary_report(df_horusec_data, get_os_windows())

# Combine all operating system reports
df_horusec_summary = pd.concat([df_horusec_macos, df_horusec_ubuntu, df_horusec_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_horusec_summary, f'{get_horusec()}_summary')

# ===================
# OWASP TOP 10 REPORT
# ===================

# Create dataframe of SAST Horusec OWASP Top 10 report
df_horusec_owasp_macos   = create_sast_owasp_report(df_horusec_data, get_os_macos())
df_horusec_owasp_ubuntu  = create_sast_owasp_report(df_horusec_data, get_os_ubuntu())
df_horusec_owasp_windows = create_sast_owasp_report(df_horusec_data, get_os_windows())

# Combine all operating system reports
df_horusec_owasp = pd.concat([df_horusec_owasp_macos, df_horusec_owasp_ubuntu, df_horusec_owasp_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_horusec_owasp, f'{get_horusec()}_owasp')

# ===================
# MITRE TOP 25 REPORT
# ===================

# Create dataframe of SAST Horusec MITRE Top 25 report
df_horusec_mitre_macos   = create_sast_mitre_report(df_horusec_data, get_os_macos())
df_horusec_mitre_ubuntu  = create_sast_mitre_report(df_horusec_data, get_os_ubuntu())
df_horusec_mitre_windows = create_sast_mitre_report(df_horusec_data, get_os_windows())

# Combine all operating system reports
df_horusec_mitre = pd.concat([df_horusec_mitre_macos, df_horusec_mitre_ubuntu, df_horusec_mitre_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_horusec_mitre, f'{get_horusec()}_mitre')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
horusec_macos_title   = f'Static Application Security Testing - {get_horusec().capitalize()} - CWE Summary ({get_os_macos()})'
horusec_ubuntu_title  = f'Static Application Security Testing - {get_horusec().capitalize()} - CWE Summary ({get_os_ubuntu()})'
horusec_windows_title = f'Static Application Security Testing - {get_horusec().capitalize()} - CWE Summary ({get_os_windows()})'

# ===== MACOS =====
horusec_summary_table_macos = render_matplotlib_table(df_horusec_macos, horusec_macos_title)

# Add separator between experiment groups
set_row_edge_color(horusec_summary_table_macos, 5,  'k')
set_row_edge_color(horusec_summary_table_macos, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_horusec()}/{get_horusec()}_summary_macos.png', bbox_inches='tight')
plt.close()

# ===== UBUNTU =====
horusec_summary_table_ubuntu = render_matplotlib_table(df_horusec_ubuntu, horusec_ubuntu_title)

# Add separator between experiment groups
set_row_edge_color(horusec_summary_table_ubuntu, 5,  'k')
set_row_edge_color(horusec_summary_table_ubuntu, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_horusec()}/{get_horusec()}_summary_ubuntu.png', bbox_inches='tight')
plt.close()

# ===== WINDOWS =====
horusec_summary_table_windows = render_matplotlib_table(df_horusec_windows, horusec_windows_title)

# Add separator between experiment groups
set_row_edge_color(horusec_summary_table_windows, 5,  'k')
set_row_edge_color(horusec_summary_table_windows, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_horusec()}/{get_horusec()}_summary_windows.png', bbox_inches='tight')
plt.close()

# =============================
# MATPLOT TABLE - OS COMPARISON
# =============================

# Create dataframe for total CWE per operating system
df_horusec_os_comparison = df_horusec_macos.filter(['Project','Tag'], axis=1)
df_horusec_os_comparison[f'Total CWE - {get_os_macos()}']   = df_horusec_macos.filter(['Total CWE'], axis=1)
df_horusec_os_comparison[f'Total CWE - {get_os_ubuntu()}']  = df_horusec_ubuntu.filter(['Total CWE'], axis=1)
df_horusec_os_comparison[f'Total CWE - {get_os_windows()}'] = df_horusec_windows.filter(['Total CWE'], axis=1)

# Drop duplicates but keep first
df_horusec_os_comparison.drop_duplicates(keep='first', inplace=True)

# Set chart titles
horusec_comparison_title = f'Static Application Security Testing - {get_horusec().capitalize()} - Runner OS Comparison'

# Create MatPlotLib table
df_horusec_os_comparison_table = render_matplotlib_table(df_horusec_os_comparison, horusec_comparison_title)

# # Add separator between experiment groups
set_row_edge_color(df_horusec_os_comparison_table, 5,  'k')
set_row_edge_color(df_horusec_os_comparison_table, 10, 'k')

# # Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_horusec()}/{get_horusec()}_os_comparison.png', bbox_inches='tight')
plt.close()

### DeepSource

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of SAST DeepSource summary report
df_deepsource_summary = create_sast_summary_report(df_deepsource_data, None)

# Write data to excel report
save_excel_sheet(df_deepsource_summary, f'{get_deepsource().lower()}_summary')

# ===================
# OWASP TOP 10 REPORT
# ===================

# Create dataframe of SAST DeepSource OWASP Top 10 report
df_deepsource_owasp = create_sast_owasp_report(df_deepsource_data, None)

# Write data to excel report
save_excel_sheet(df_deepsource_owasp, f'{get_deepsource().lower()}_owasp')

# ===================
# MITRE TOP 25 REPORT
# ===================

# Create dataframe of SAST DeepSource MITRE Top 25 report
df_deepsource_mitre = create_sast_mitre_report(df_deepsource_data, None)

# Write data to excel report
save_excel_sheet(df_deepsource_mitre, f'{get_deepsource().lower()}_mitre')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
deepsource_title = f'Static Application Security Testing - {get_deepsource()} - CWE Summary'

deepsource_summary_table = render_matplotlib_table(df_deepsource_summary, deepsource_title)

# Add separator between experiment groups
set_row_edge_color(deepsource_summary_table, 5,  'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_deepsource().lower()}/{get_deepsource().lower()}_summary.png', bbox_inches='tight')
plt.close()

### Semgrep

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of SAST Semgrep summary report
df_semgrep_macos   = create_sast_summary_report(df_semgrep_data, get_os_macos())
df_semgrep_ubuntu  = create_sast_summary_report(df_semgrep_data, get_os_ubuntu())
df_semgrep_windows = create_sast_summary_report(df_semgrep_data, get_os_windows())

# Combine all operating system reports
df_semgrep_summary = pd.concat([df_semgrep_macos, df_semgrep_ubuntu, df_semgrep_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_semgrep_summary, f'{get_semgrep()}_summary')

# ===================
# OWASP TOP 10 REPORT
# ===================

# Create dataframe of SAST Semgrep OWASP Top 10 report
df_semgrep_owasp_macos   = create_sast_owasp_report(df_semgrep_data, get_os_macos())
df_semgrep_owasp_ubuntu  = create_sast_owasp_report(df_semgrep_data, get_os_ubuntu())
df_semgrep_owasp_windows = create_sast_owasp_report(df_semgrep_data, get_os_windows())

# Combine all operating system reports
df_semgrep_owasp = pd.concat([df_semgrep_owasp_macos, df_semgrep_owasp_ubuntu, df_semgrep_owasp_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_semgrep_owasp, f'{get_semgrep()}_owasp')

# ===================
# MITRE TOP 25 REPORT
# ===================

# Create dataframe of SAST Semgrep MITRE Top 25 report
df_semgrep_mitre_macos   = create_sast_mitre_report(df_semgrep_data, get_os_macos())
df_semgrep_mitre_ubuntu  = create_sast_mitre_report(df_semgrep_data, get_os_ubuntu())
df_semgrep_mitre_windows = create_sast_mitre_report(df_semgrep_data, get_os_windows())

# Combine all operating system reports
df_semgrep_mitre = pd.concat([df_semgrep_mitre_macos, df_semgrep_mitre_ubuntu, df_semgrep_mitre_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_semgrep_mitre, f'{get_semgrep()}_mitre')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
semgrep_macos_title   = f'Static Application Security Testing - {get_semgrep().capitalize()} - CWE Summary ({get_os_macos()})'
semgrep_ubuntu_title  = f'Static Application Security Testing - {get_semgrep().capitalize()} - CWE Summary ({get_os_ubuntu()})'
semgrep_windows_title = f'Static Application Security Testing - {get_semgrep().capitalize()} - CWE Summary ({get_os_windows()})'

# ===== MACOS =====
semgrep_summary_table_macos = render_matplotlib_table(df_semgrep_macos, semgrep_macos_title)

# Add separator between experiment groups
set_row_edge_color(semgrep_summary_table_macos, 5,  'k')
set_row_edge_color(semgrep_summary_table_macos, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_semgrep()}/{get_semgrep()}_summary_macos.png', bbox_inches='tight')
plt.close()

# ===== UBUNTU =====
semgrep_summary_table_ubuntu = render_matplotlib_table(df_semgrep_ubuntu, semgrep_ubuntu_title)

# Add separator between experiment groups
set_row_edge_color(semgrep_summary_table_ubuntu, 5,  'k')
set_row_edge_color(semgrep_summary_table_ubuntu, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_semgrep()}/{get_semgrep()}_summary_ubuntu.png', bbox_inches='tight')
plt.close()

# ===== WINDOWS =====
semgrep_summary_table_windows = render_matplotlib_table(df_semgrep_windows, semgrep_windows_title)

# Add separator between experiment groups
set_row_edge_color(semgrep_summary_table_windows, 5,  'k')
set_row_edge_color(semgrep_summary_table_windows, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_semgrep()}/{get_semgrep()}_summary_windows.png', bbox_inches='tight')
plt.close()

# =============================
# MATPLOT TABLE - OS COMPARISON
# =============================

# Create dataframe for total CWE per operating system
df_semgrep_os_comparison = df_semgrep_macos.filter(['Project','Tag'], axis=1)
df_semgrep_os_comparison[f'Total CWE - {get_os_macos()}']   = df_semgrep_macos.filter(['Total CWE'], axis=1)
df_semgrep_os_comparison[f'Total CWE - {get_os_ubuntu()}']  = df_semgrep_ubuntu.filter(['Total CWE'], axis=1)
df_semgrep_os_comparison[f'Total CWE - {get_os_windows()}'] = df_semgrep_windows.filter(['Total CWE'], axis=1)

# Drop duplicates but keep first
df_semgrep_os_comparison.drop_duplicates(keep='first', inplace=True)

# Set chart titles
semgrep_comparison_title = f'Static Application Security Testing - {get_semgrep().capitalize()} - Runner OS Comparison'

# Create MatPlotLib table
df_semgrep_os_comparison_table = render_matplotlib_table(df_semgrep_os_comparison, semgrep_comparison_title)

# # Add separator between experiment groups
set_row_edge_color(df_semgrep_os_comparison_table, 5,  'k')
set_row_edge_color(df_semgrep_os_comparison_table, 10, 'k')

# # Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_semgrep()}/{get_semgrep()}_os_comparison.png', bbox_inches='tight')
plt.close()

### Snyk Code

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

snyk_code_name = get_snyk_code().replace(" ","_")

# Create dataframe of SAST Snyk Code summary report
df_snyk_code_macos   = create_sast_summary_report(df_snyk_code_data, get_os_macos())
df_snyk_code_ubuntu  = create_sast_summary_report(df_snyk_code_data, get_os_ubuntu())
df_snyk_code_windows = create_sast_summary_report(df_snyk_code_data, get_os_windows())

# Combine all operating system reports
df_snyk_code_summary = pd.concat([df_snyk_code_macos, df_snyk_code_ubuntu, df_snyk_code_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_snyk_code_summary, f'{snyk_code_name}_summary')

# ===================
# OWASP TOP 10 REPORT
# ===================

# Create dataframe of SAST Snyk Code OWASP Top 10 report
df_snyk_code_owasp_macos   = create_sast_owasp_report(df_snyk_code_data, get_os_macos())
df_snyk_code_owasp_ubuntu  = create_sast_owasp_report(df_snyk_code_data, get_os_ubuntu())
df_snyk_code_owasp_windows = create_sast_owasp_report(df_snyk_code_data, get_os_windows())

# Combine all operating system reports
df_snyk_code_owasp = pd.concat([df_snyk_code_owasp_macos, df_snyk_code_owasp_ubuntu, df_snyk_code_owasp_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_snyk_code_owasp, f'{snyk_code_name}_owasp')

# ===================
# MITRE TOP 25 REPORT
# ===================

# Create dataframe of SAST Snyk Code MITRE Top 25 report
df_snyk_code_mitre_macos   = create_sast_mitre_report(df_snyk_code_data, get_os_macos())
df_snyk_code_mitre_ubuntu  = create_sast_mitre_report(df_snyk_code_data, get_os_ubuntu())
df_snyk_code_mitre_windows = create_sast_mitre_report(df_snyk_code_data, get_os_windows())

# Combine all operating system reports
df_snyk_code_mitre = pd.concat([df_snyk_code_mitre_macos, df_snyk_code_mitre_ubuntu, df_snyk_code_mitre_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_snyk_code_mitre, f'{snyk_code_name}_mitre')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
snyk_code_macos_title   = f'Static Application Security Testing - {get_snyk_code().title()} - CWE Summary ({get_os_macos()})'
snyk_code_ubuntu_title  = f'Static Application Security Testing - {get_snyk_code().title()} - CWE Summary ({get_os_ubuntu()})'
snyk_code_windows_title = f'Static Application Security Testing - {get_snyk_code().title()} - CWE Summary ({get_os_windows()})'

# ===== MACOS =====
snyk_code_summary_table_macos = render_matplotlib_table(df_snyk_code_macos, snyk_code_macos_title)

# Add separator between experiment groups
set_row_edge_color(snyk_code_summary_table_macos, 2,  'k')
set_row_edge_color(snyk_code_summary_table_macos, 5, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{snyk_code_name}/{snyk_code_name}_summary_macos.png', bbox_inches='tight')
plt.close()

# ===== UBUNTU =====
snyk_code_summary_table_ubuntu = render_matplotlib_table(df_snyk_code_ubuntu, snyk_code_ubuntu_title)

# Add separator between experiment groups
set_row_edge_color(snyk_code_summary_table_ubuntu, 2,  'k')
set_row_edge_color(snyk_code_summary_table_ubuntu, 5, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{snyk_code_name}/{snyk_code_name}_summary_ubuntu.png', bbox_inches='tight')
plt.close()

# ===== WINDOWS =====
snyk_code_summary_table_windows = render_matplotlib_table(df_snyk_code_windows, snyk_code_windows_title)

# Add separator between experiment groups
set_row_edge_color(snyk_code_summary_table_windows, 2,  'k')
set_row_edge_color(snyk_code_summary_table_windows, 5, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{snyk_code_name}/{snyk_code_name}_summary_windows.png', bbox_inches='tight')
plt.close()

# =============================
# MATPLOT TABLE - OS COMPARISON
# =============================

# Create dataframe for total CWE per operating system
df_snyk_code_os_comparison = df_snyk_code_macos.filter(['Project','Tag'], axis=1)
df_snyk_code_os_comparison[f'Total CWE - {get_os_macos()}']   = df_snyk_code_macos.filter(['Total CWE'], axis=1)
df_snyk_code_os_comparison[f'Total CWE - {get_os_ubuntu()}']  = df_snyk_code_ubuntu.filter(['Total CWE'], axis=1)
df_snyk_code_os_comparison[f'Total CWE - {get_os_windows()}'] = df_snyk_code_windows.filter(['Total CWE'], axis=1)

# Drop duplicates but keep first
df_snyk_code_os_comparison.drop_duplicates(keep='first', inplace=True)

# Set chart titles
snyk_code_comparison_title = f'Static Application Security Testing - {get_snyk_code().title()} - Runner OS Comparison'

# Create MatPlotLib table
df_snyk_code_os_comparison_table = render_matplotlib_table(df_snyk_code_os_comparison, snyk_code_comparison_title)

# # Add separator between experiment groups
set_row_edge_color(df_snyk_code_os_comparison_table, 2,  'k')
set_row_edge_color(df_snyk_code_os_comparison_table, 5, 'k')

# # Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{snyk_code_name}/{snyk_code_name}_os_comparison.png', bbox_inches='tight')
plt.close()

### SonarQube

In [ ]:
# ==============
# SUMMARY REPORT
# ==============

# Create dataframe of SAST SonarQube summary report
df_sonarqube_macos   = create_sast_summary_report(df_sonarqube_data, get_os_macos())
df_sonarqube_ubuntu  = create_sast_summary_report(df_sonarqube_data, get_os_ubuntu())
df_sonarqube_windows = create_sast_summary_report(df_sonarqube_data, get_os_windows())

# Combine all operating system reports
df_sonarqube_summary = pd.concat([df_sonarqube_macos, df_sonarqube_ubuntu, df_sonarqube_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_sonarqube_summary, f'{get_sonarqube().lower()}_summary')

# ===================
# OWASP TOP 10 REPORT
# ===================

# Create dataframe of SAST SonarQube OWASP Top 10 report
df_sonarqube_owasp_macos   = create_sast_owasp_report(df_sonarqube_data, get_os_macos())
df_sonarqube_owasp_ubuntu  = create_sast_owasp_report(df_sonarqube_data, get_os_ubuntu())
df_sonarqube_owasp_windows = create_sast_owasp_report(df_sonarqube_data, get_os_windows())

# Combine all operating system reports
df_sonarqube_owasp = pd.concat([df_sonarqube_owasp_macos, df_sonarqube_owasp_ubuntu, df_sonarqube_owasp_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_sonarqube_owasp, f'{get_sonarqube().lower()}_owasp')

# ===================
# MITRE TOP 25 REPORT
# ===================

# Create dataframe of SAST SonarQube MITRE Top 25 report
df_sonarqube_mitre_macos   = create_sast_mitre_report(df_sonarqube_data, get_os_macos())
df_sonarqube_mitre_ubuntu  = create_sast_mitre_report(df_sonarqube_data, get_os_ubuntu())
df_sonarqube_mitre_windows = create_sast_mitre_report(df_sonarqube_data, get_os_windows())

# Combine all operating system reports
df_sonarqube_mitre = pd.concat([df_sonarqube_mitre_macos, df_sonarqube_mitre_ubuntu, df_sonarqube_mitre_windows], ignore_index=True, sort=False)

# Write data to excel report
save_excel_sheet(df_sonarqube_mitre, f'{get_sonarqube().lower()}_mitre')

# =======================
# MATPLOT TABLE - SUMMARY
# =======================

# Set chart titles
sonarqube_macos_title   = f'Static Application Security Testing - {get_sonarqube()} - CWE Summary ({get_os_macos()})'
sonarqube_ubuntu_title  = f'Static Application Security Testing - {get_sonarqube()} - CWE Summary ({get_os_ubuntu()})'
sonarqube_windows_title = f'Static Application Security Testing - {get_sonarqube()} - CWE Summary ({get_os_windows()})'

# ===== MACOS =====
sonarqube_summary_table_macos = render_matplotlib_table(df_sonarqube_macos, sonarqube_macos_title)

# Add separator between experiment groups
set_row_edge_color(sonarqube_summary_table_macos, 5,  'k')
set_row_edge_color(sonarqube_summary_table_macos, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_sonarqube().lower()}/{get_sonarqube().lower()}_summary_macos.png', bbox_inches='tight')
plt.close()

# ===== UBUNTU =====
sonarqube_summary_table_ubuntu = render_matplotlib_table(df_sonarqube_ubuntu, sonarqube_ubuntu_title)

# Add separator between experiment groups
set_row_edge_color(sonarqube_summary_table_ubuntu, 5,  'k')
set_row_edge_color(sonarqube_summary_table_ubuntu, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_sonarqube().lower()}/{get_sonarqube().lower()}_summary_ubuntu.png', bbox_inches='tight')
plt.close()

# ===== WINDOWS =====
sonarqube_summary_table_windows = render_matplotlib_table(df_sonarqube_windows, sonarqube_windows_title)

# Add separator between experiment groups
set_row_edge_color(sonarqube_summary_table_windows, 5,  'k')
set_row_edge_color(sonarqube_summary_table_windows, 10, 'k')

# Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_sonarqube().lower()}/{get_sonarqube().lower()}_summary_windows.png', bbox_inches='tight')
plt.close()

# =============================
# MATPLOT TABLE - OS COMPARISON
# =============================

# Create dataframe for total CWE per operating system
df_sonarqube_os_comparison = df_sonarqube_macos.filter(['Project','Tag'], axis=1)
df_sonarqube_os_comparison[f'Total CWE - {get_os_macos()}']   = df_sonarqube_macos.filter(['Total CWE'], axis=1)
df_sonarqube_os_comparison[f'Total CWE - {get_os_ubuntu()}']  = df_sonarqube_ubuntu.filter(['Total CWE'], axis=1)
df_sonarqube_os_comparison[f'Total CWE - {get_os_windows()}'] = df_sonarqube_windows.filter(['Total CWE'], axis=1)

# Drop duplicates but keep first
df_sonarqube_os_comparison.drop_duplicates(keep='first', inplace=True)

# Set chart titles
sonarqube_comparison_title = f'Static Application Security Testing - {get_sonarqube()} - Runner OS Comparison'

# Create MatPlotLib table
df_sonarqube_os_comparison_table = render_matplotlib_table(df_sonarqube_os_comparison, sonarqube_comparison_title)

# # Add separator between experiment groups
set_row_edge_color(df_sonarqube_os_comparison_table, 5,  'k')
set_row_edge_color(df_sonarqube_os_comparison_table, 10, 'k')

# # Save chart as image
plt.savefig(f'{get_notebook_directory_path_prefix()}/tables/{current_date}/static_application_security_testing/{get_sonarqube().lower()}/{get_sonarqube().lower()}_os_comparison.png', bbox_inches='tight')
plt.close()